In [1]:
import os
import ssl
import urllib3

# 1. Tắt cảnh báo bảo mật (để đỡ rối mắt)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# 2. Xóa các biến trỏ file sai
os.environ.pop("AWS_CA_BUNDLE", None)
os.environ.pop("CURL_CA_BUNDLE", None)

# 3. Ép buộc tắt SSL Verification ở mức Environment
os.environ["MLFLOW_TRACKING_INSECURE_TLS"] = "true"
os.environ["AWS_NO_SIGN_REQUEST"] = "NO" # Đảm bảo vẫn ký request nhưng không verify SSL

# 4. "Vá" module SSL của Python (Monkey Patching)
# Đoạn này ép Python bỏ qua lỗi certificate cho mọi kết nối HTTPS
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

print("Đã tắt toàn bộ kiểm tra SSL/TLS. Mọi kết nối sẽ không an toàn nhưng sẽ chạy được.")

Đã tắt toàn bộ kiểm tra SSL/TLS. Mọi kết nối sẽ không an toàn nhưng sẽ chạy được.


In [3]:
import mlflow
mlflow.set_tracking_uri("http://ec2-16-176-5-94.ap-southeast-2.compute.amazonaws.com:5000")

In [4]:
mlflow.set_experiment("Exp 4 - Handling Imbalanced Data")

<Experiment: artifact_location='s3://amzn-s3-mlflow-bucket/4', creation_time=1763706944223, experiment_id='4', last_update_time=1763706944223, lifecycle_stage='active', name='Exp 4 - Handling Imbalanced Data', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [5]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [6]:
df = pd.read_csv('../data/reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [7]:
# Step 1: Function to run the experiment
def run_imbalanced_experiment(imbalance_method):
    ngram_range = (1, 3)  # Trigram setting
    max_features = 1000  # Set max_features to 1000 for TF-IDF

    # Step 4: Train-test split before vectorization and resampling
    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    # Step 2: Vectorization using TF-IDF, fit on training data only
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
    X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
    X_test_vec = vectorizer.transform(X_test)  # Transform test data

    # Step 3: Handle class imbalance based on the selected method (only applied to the training set)
    if imbalance_method == 'class_weights':
        # Use class_weight in Random Forest
        class_weight = 'balanced'
    else:
        class_weight = None  # Do not apply class_weight if using resampling

        # Resampling Techniques (only apply to the training set)
        if imbalance_method == 'oversampling':
            smote = SMOTE(random_state=42)
            X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'adasyn':
            adasyn = ADASYN(random_state=42)
            X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'undersampling':
            rus = RandomUnderSampler(random_state=42)
            X_train_vec, y_train = rus.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'smote_enn':
            smote_enn = SMOTEENN(random_state=42)
            X_train_vec, y_train = smote_enn.fit_resample(X_train_vec, y_train)

    # Step 5: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"Imbalance_{imbalance_method}_RandomForest_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "imbalance_handling")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, imbalance handling method={imbalance_method}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("imbalance_method", imbalance_method)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42, class_weight=class_weight)
        model.fit(X_train_vec, y_train)

        # Step 6: Make predictions and log metrics
        y_pred = model.predict(X_test_vec)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, Imbalance={imbalance_method}")
        confusion_matrix_filename = f"../training_result/confusion_matrix_{imbalance_method}.png"
        plt.savefig(confusion_matrix_filename)
        mlflow.log_artifact(confusion_matrix_filename)
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_imbalance_{imbalance_method}")

# Step 7: Run experiments for different imbalance methods
imbalance_methods = ['class_weights', 'oversampling', 'adasyn', 'undersampling', 'smote_enn']

for method in imbalance_methods:
    run_imbalanced_experiment(method)

2025/11/21 14:59:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/21 15:00:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_class_weights_RandomForest_TFIDF_Trigrams at: http://ec2-16-176-5-94.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/4/runs/0ecde26ebaab4f5ebcb5ec3d057a1460
🧪 View experiment at: http://ec2-16-176-5-94.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/4


2025/11/21 15:01:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/21 15:01:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_oversampling_RandomForest_TFIDF_Trigrams at: http://ec2-16-176-5-94.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/4/runs/a43bc58fa506473bbe13f8b940897438
🧪 View experiment at: http://ec2-16-176-5-94.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/4


2025/11/21 15:03:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/21 15:03:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_adasyn_RandomForest_TFIDF_Trigrams at: http://ec2-16-176-5-94.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/4/runs/441427b70a8243188a594948395487e7
🧪 View experiment at: http://ec2-16-176-5-94.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/4


2025/11/21 15:05:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/21 15:05:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_undersampling_RandomForest_TFIDF_Trigrams at: http://ec2-16-176-5-94.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/4/runs/7d3d1920352c487a925b5a6ac1ff82c7
🧪 View experiment at: http://ec2-16-176-5-94.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/4


2025/11/21 15:07:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/21 15:07:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_smote_enn_RandomForest_TFIDF_Trigrams at: http://ec2-16-176-5-94.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/4/runs/2f1e6e661dad44ecb47dd24c14f88210
🧪 View experiment at: http://ec2-16-176-5-94.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/4
